In [72]:
# Dependencies
import pandas as pd
from bs4 import BeautifulSoup
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

## NASA Mars News

In [73]:
# Splinter browser initialization
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\pupor\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


In [74]:
# Import URL
url = "https://redplanetscience.com/#"

In [75]:
# Retrieve page using the browser
browser.visit(url)

In [76]:
# Create HTML object and parse with html parser
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [77]:
# Store contents of divs containing title and paragraph
title = soup.find('div', class_ = 'content_title').text
paragraph = soup.find('div', class_ = 'article_teaser_body').text

## JPL Mars Space Images - Featured Image

In [78]:
# URL 
url = 'https://spaceimages-mars.com/'

In [79]:
# Visit using browser
browser.visit(url)

In [80]:
# Retrieve html

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [81]:
# Retrieve image data from the html parser and store in a variable

image_url = soup.find('a', class_ = "showimg fancybox-thumbs")['href']
featured_image_url = f"{url}{image_url}"

## Mars Facts

In [82]:
# Define URL and read using Pandas
url = 'https://galaxyfacts-mars.com/'
tables = pd.read_html(url)

In [83]:
# Extract table and transform to tidy up

mars_facts_table = tables[0]
mars_facts_table.columns = mars_facts_table.iloc[0]
mars_facts_table = mars_facts_table.iloc[1:, ]
mars_facts_table.set_index("Mars - Earth Comparison", inplace = True)
mars_facts_table

,Mars,Earth
Mars - Earth Comparison,,
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [84]:
# Convert to HTML

html_table = mars_facts_table.to_html()

## Mars Hemispheres Data

In [85]:
# Set up URL of interest and visit using browser

url = 'https://marshemispheres.com/'
browser.visit(url)

In [86]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [87]:
hemi_list = []
hemis = soup.find_all('h3')
hemis

for hemi in hemis:
    hemi_name = hemi.text
    hemi_list.append(hemi_name)

del hemi_list[len(hemi_list) - 1]

In [88]:
# Hemisphere list to store dictionaries in

hemisphere_image_urls = []

# For loop to retrieve image URL and hemisphere name
for hemi in hemi_list:
    
    # Retrieve html to know what I'm scraping for
    browser.links.find_by_partial_text(hemi).click()
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    hemisphere_dict = {}

    # Extract the title
    hemi_title = soup.find('div', class_ = 'cover')
    hemi_title = hemi_title.h2.text.split(' ')[:-1]
    hemi_title = " ".join(hemi_title)
    hemisphere_dict['title'] = hemi_title

    # Extract image url
    image_url = soup.find('ul').li.a['href']
    hemisphere_dict['img_url'] = f"{url}{image_url}"
    
    # Append dict to list
    hemisphere_image_urls.append(hemisphere_dict)
    
    # Go back to main page 
    browser.links.find_by_partial_text('Back').click()

hemisphere_image_urls

[{'title': 'Cerberus Hemisphere',
  'img_url': 'https://marshemispheres.com/images/full.jpg'},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg'},
 {'title': 'Valles Marineris Hemisphere',
  'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg'}]

In [89]:
# Quit browser instance
browser.quit()